<a href="https://colab.research.google.com/github/chipojaya1/Machine-Learning-I/blob/main/Practical%20Assignment%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DNSC6314 Assignment 1 by Chipo Jaya: G44454879**

## This assignment uses Capital Bikeshare Data
https://ride.capitalbikeshare.com/system-data

Data is from three months: 2024/02, 2024/03 and 2024/04.

### Getting started

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# cloning github repository to access data
!git clone https://github.com/chipojaya1/Machine-Learning-I.git

Cloning into 'Machine-Learning-I'...
fatal: could not read Username for 'https://github.com': No such device or address


In [3]:
# load necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for feature engineering
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
sns.set(style='whitegrid')     # Set visual style for seaborn

In [ ]:
# load the data
df_Feb = pd.read_csv('202402-capitalbikeshare-tripdata.csv')
df_Mar = pd.read_csv('202403-capitalbikeshare-tripdata.csv')
df_Apr = pd.read_csv('202404-capitalbikeshare-tripdata.csv')

# concat data
df_bike=pd.concat([df_Feb, df_Mar,df_Apr])

In [ ]:
df_bike.shape

In [ ]:
df_bike.head()

In [ ]:
df_bike.info()

## Focusing on the GWSB Station: '22nd & H St NW'

### Bike Availability: Number of Pickups

In [ ]:
# Convert the 'started_at' column to datetime objects
df_bike['started_at_date'] = pd.to_datetime(df_bike['started_at']).dt.date

# Filter for trips started at '22nd & H St NW'
PU_trips = df_bike[df_bike['start_station_name'] == '22nd & H St NW']

# Group by date and count the trips
PU_counts = PU_trips.groupby('started_at_date')['started_at_date'].count()

PU_counts

## Dock Availability: Number of Dropoffs

In [ ]:
# Convert the 'ended_at' column to datetime objects
df_bike['ended_at_date'] = pd.to_datetime(df_bike['ended_at']).dt.date

# Filter for trips ended at '22nd & H St NW'
DO_trips = df_bike[df_bike['end_station_name'] == '22nd & H St NW']

# Group by date and count the trips
DO_counts = DO_trips.groupby('ended_at_date')['ended_at_date'].count()

DO_counts

## Merging Pick Up and Drop Off tables
- For each day, we look at the numbers of pickups and dropoffs

In [ ]:
# merge PU_counts and DO_counts by matching the dates, and change the column names to PU_count and DO_count respectively.

PU_DO_counts = pd.merge(PU_counts, DO_counts, left_index=True, right_index=True, how='outer')
PU_DO_counts = PU_DO_counts.rename(columns={'started_at_date': 'PU_ct', 'ended_at_date': 'DO_ct'})

# change the index name from 'started_at' to 'date'

PU_DO_counts = PU_DO_counts.rename_axis('date')
PU_DO_counts


## Visualizing the pickups and dropoffs

In [ ]:
# plot line chart showing the PU_count and DO_count over time
import matplotlib.pyplot as plt

# Create the line chart
plt.figure(figsize=(12, 6))
plt.plot(PU_DO_counts.index, PU_DO_counts['PU_ct'], label='PU_ct')
plt.plot(PU_DO_counts.index, PU_DO_counts['DO_ct'], label='DO_ct')

# Customize the chart
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Pickups and Dropoffs Over Time')
plt.legend()
plt.grid(True)

plt.show()

# Feature Information: Weather Data
- From https://www.visualcrossing.com/weather-history/

In [ ]:
# Loading the weather data
df_weather = pd.read_csv('DC_weather_2024.csv')

# Inspecting the dataframe
df_weather.head()

In [ ]:
# Checking info of columns
df_weather.info()

## Drop variables of your choice

In [ ]:
# Drop unnecessary variables
df_weather=df_weather.drop(columns=['name', 'stations','description','sunrise','sunset','conditions','severerisk','preciptype','windgust'])

## Prepare X and y: Merge df_weather with PU_DO_counts

In [ ]:
# Convert the 'datetime' column to datetime objects and extract the date
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])
df_weather['date'] = df_weather['datetime'].dt.date

# Merge the two dataframes based on the 'date' column
merged_df = pd.merge(PU_DO_counts, df_weather, on='date', how='left')

# Display the merged dataframe
merged_df

In [ ]:
import seaborn as sns
sns.pairplot(merged_df[['PU_ct',"temp", "precip", "windspeed","uvindex"]], kind="reg",plot_kws=dict(scatter_kws=dict(s=2), line_kws = {'color':'black'})) # pairplot for PU_ct

# Prepare training and test data

In [ ]:
# we have two target variables: PU_ct and DO_ct
y = merged_df[['PU_ct','DO_ct']]
X = merged_df[['temp','precip','windspeed','uvindex','icon']]
X

# **Assignment 1: Regression Models**

## Task 1: Train a linear regression model with a single feature ('temp') for PU_ct, and report the training and test MSE, respectively. [10 pts]

In [ ]:
# Preparing the training and test data
y = merged_df['PU_ct']
X = merged_df[['temp']]
X

In [ ]:
# Splitting the data: 60% train data and 40% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=200)

In [ ]:
# Training the linear regression model:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Predicting and calculating MSE:
model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

In [ ]:
# Storing MSE for Task 1 features: temp
train_mse_task1 = train_mse
test_mse_task1 = test_mse

## Task 2: Train a linear regression model with two feature ('temp' and 'precip') for PU_ct, and report the training and test MSE, respectively. [10 pts]

In [ ]:
# Preparing the training and test data
y = merged_df['PU_ct']
X = merged_df[['temp', 'precip']]
X

In [ ]:
# Splitting the data: 60% train data and 40% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=200)

In [ ]:
# Training the linear regression model:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Predicting and calculating MSE:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

In [ ]:
# Storing MSE for Task 2 features: temp + precip
train_mse_task2 = train_mse
test_mse_task2 = test_mse

## Task 3: Train a linear regression model with features ('temp','precip','windspeed') for PU_ct, and report the training and test MSE, respectively. [10 pts]

In [ ]:
# Preparing the training and test data
y = merged_df['PU_ct']
X = merged_df[['temp', 'precip', 'windspeed']]
X

In [ ]:
# Splitting the data: 60% train data and 40% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=200)

In [ ]:
# Training the linear regression model:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Predicting and calculating MSE:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

In [ ]:
# Storing MSE for Task 3 featuers: temp + precip + windspeed
train_mse_task3 = train_mse
test_mse_task3 = test_mse

## Task 4: Train a linear regression model with features ('temp','precip','windspeed','uvindex') for PU_ct, and report the training and test MSE, respectively. [10 pts]

In [ ]:
# Preparing the training and test data
y = merged_df['PU_ct']
X = merged_df[['temp', 'precip', 'windspeed', 'uvindex']]
X

In [ ]:
# Splitting the data: 60% train data and 40% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=200)

In [ ]:
# Training the linear regression model:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Predicting and calculating MSE:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

In [ ]:
# Storing MSE for Task 4 features: temp + precip + windspeed + uvindex
train_mse_task4 = train_mse
test_mse_task4 = test_mse

## Task 5: Train a linear regression model with features ('temp','precip','windspeed','uvindex','icon') for PU_ct, and report the training and test MSE, respectively. [10 pts]

In [ ]:
# Preparing the training and test data
y = merged_df['PU_ct']
X = pd.get_dummies(merged_df[['temp', 'precip', 'windspeed', 'uvindex', 'icon']], columns=['icon'], drop_first=True)
X

In [ ]:
# Splitting the data: 60% train data and 40% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=200)

In [ ]:
# Training the linear regression model:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Predicting and calculating MSE:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

In [ ]:
# Storing MSE for Task 5 features: temp + precip + windspeed + uvindex + icon_dummies
train_mse_task5 = train_mse
test_mse_task5 = test_mse

## Task 6: Based on the previous results, plot the changes in training and test MSEs as more features are added to the linear regression model. [10 pts]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# MSE values from the Tasks 1–5
mse_data = {
    'Features': [
        'temp',
        'temp,precip',
        'temp,precip,windspeed',
        'temp,precip,windspeed,uvindex',
        'temp,precip,windspeed,uvindex,icon_dummies'
    ],
    'Train_MSE': [train_mse_task1, train_mse_task2, train_mse_task3, train_mse_task4, train_mse_task5],
    'Test_MSE': [test_mse_task1, test_mse_task2, test_mse_task3, test_mse_task4, test_mse_task5]
}
# DataFrame storing the MSE values
mse_df = pd.DataFrame(mse_data)

In [ ]:
# Plotting the train and test MSE values
plt.figure(figsize=(8, 5))
plt.plot(mse_df['Features'], mse_df['Train_MSE'], label='Train MSE', marker='o')
plt.plot(mse_df['Features'], mse_df['Test_MSE'], label='Test MSE', marker='s')
plt.xlabel('Features')
plt.ylabel('MSE')
plt.title('Training and Test MSE vs. Features')
plt.legend()
plt.grid(True)
plt.xticks(rotation=60)  # Rotating x-axis labels for better readability
plt.show()

## Task 7: Based on the above plot, identify the best combination of features for PU_ct prediction in linear regression. [10 pts]

## Task 8: Conduct Task 1-7 again for DO_ct prediction. [30 pts]

#Reference
##### <font color="red">"DeepSeek AI (Version 1.0, developed by DeepSeek AI, accessed January 28, 2025)" was consulted during the research process.</font>
